In [513]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import pandas as pd

In [514]:
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
sns.set()

# CUSTOM LAYERS

In [515]:
class InputLayer(tf.keras.layers.Layer):
  def __init__(self, name):
    super(InputLayer, self).__init__(name=name)
    self.a = tf.Variable(initial_value=tf.zeros(shape=[2,14]), trainable=False, name= self.name + "_activation")
    
  def call(self, inputs):
    a1a1 = tf.reshape(inputs[0],shape=(1,1))   #1
    a1a2 = tf.reshape(inputs[1],shape=(1,1))   #2 
    a1a2a2 = tf.reshape(inputs[2],shape=(1,1)) #3
    a1a2a1 = tf.reshape(inputs[3],shape=(1,1)) #4
    a2a2 = tf.reshape(inputs[4],shape=(1,1))   #5
    a2a1 = tf.reshape(inputs[5],shape=(1,1))   #6
    a2a1a1 = tf.reshape(inputs[6],shape=(1,1)) #7
    a2a1a2 = tf.reshape(inputs[7],shape=(1,1)) #8

    to_hidden = []
    to_hidden.append((tf.concat([a1a2a2, a2a2], axis = 1)))   #["TrRi"] [3,5]
    to_hidden.append((tf.concat([a1a2a1, a2a1], axis = 1)))   #["RiTr"] [4,6] 
    to_hidden.append((tf.concat([a1a1, a2a1], axis = 1)))     #["Plu"]  [1,6] 
    to_hidden.append((tf.concat([a1a2, a1a2a2], axis = 1)))   #["TTra"] [2,3] 
    to_hidden.append((tf.concat([a2a1, a2a1a1], axis = 1)))   #["TTrb"] [6,7] 
    to_hidden.append((tf.concat([a1a2, a2a1], axis = 1)))     #["Tr"]   [2,6]
    to_hidden.append((tf.concat([a1a2a1, a2a1a2], axis = 1))) #["MeTr"] [4,8] 
    to_hidden.append((tf.concat([a1a2a1, a1a2], axis = 1)))   #["MoTr"] [4,2] 
    to_hidden.append((tf.concat([a1a1, a1a2a1], axis = 1)))   #["RiRi"] [1,4] 
    to_hidden.append((tf.concat([a1a1, a1a2], axis = 1)))     #["MoFo"] [1,2] 
    to_hidden.append((tf.concat([a1a2a2, a1a2a1], axis = 1))) #["MMof"] [3,4] 
    to_hidden.append((tf.concat([a1a1, a1a2a2], axis = 1)))   #["MoRi"] [1,3] 
    to_hidden.append((tf.concat([a1a2a2, a2a1a1], axis = 1))) #["MeRi"] [3,7] 
    to_hidden.append((tf.concat([a1a1, a2a2], axis = 1)))     #["Ri"]   [1,5] 

    output = tf.concat(to_hidden,axis = 0)
    output = tf.transpose(output)
    self.a.assign(output)
    return self.a.value()

In [516]:
class HiddenLayer(tf.keras.layers.Layer):
  def __init__(self, name, weights_init_val, bias_init_val):
    super(HiddenLayer, self).__init__(name = name)
    self.units = 14
    self.weights_init_val = weights_init_val
    self.bias_init_val = bias_init_val

  def build(self, input_shape):
    shape = [int(input_shape[0]), self.units]
    self.w = self.add_weight(initializer = tf.initializers.Constant(self.weights_init_val) ,
                              shape= shape,
                              name = self.name + "_weights")
    self.b = self.add_weight(initializer = tf.initializers.Constant(self.bias_init_val) ,
                              shape= [1, self.units],
                              name = self.name + "_biases")

    self.deltaA = tf.Variable(initial_value=tf.zeros(shape = [1, self.units]),trainable=False, name= self.name + "_adjustements")
    self.a = tf.Variable(initial_value=tf.zeros(shape = [1, self.units]),trainable=False, name= self.name + "_activation")

  def call(self, inputs, temperature):
     # i pesi dei neuroni sono considerati in colonna
    net = tf.math.reduce_sum(inputs * self.w, axis=0) 
    self.a.assign( 1/( 1 + tf.exp( -( (net + self.b)/temperature)) ) )
    return self.a.value()


In [517]:
class OutputLayer(tf.keras.layers.Layer):
  def __init__(self, name, weights_init_val, bias_init_val):
    super(OutputLayer, self).__init__(name = name)
    self.units = 14
    self.weights_init_val = weights_init_val
    self.bias_init_val = bias_init_val

  def build(self, input_shape):
    shape = [int(input_shape[-1]), self.units]
    self.w = self.add_weight(initializer = tf.initializers.Constant(self.weights_init_val) ,
                              shape= shape,
                              name = self.name + "_weights")
    self.b = self.add_weight(initializer = tf.initializers.Constant(self.bias_init_val) ,
                              shape= [1, self.units],
                              name = self.name + "_biases")

    self.deltaA = tf.Variable(initial_value=tf.zeros(shape = [1, self.units]),trainable=False, name= self.name + "_adjustements")
    self.a = tf.Variable(initial_value=tf.zeros(shape = [1, self.units]),trainable=False, name= self.name + "_activation")

  def call(self, inputs, temperature):
     # i pesi dei neuroni sono considerati in colonna
     
    net = tf.matmul(inputs,self.w) 
    self.a.assign( 1/( 1 + tf.exp( -( (net + self.b)/temperature)) ) )
    return self.a.value()

### Testing foward pass

In [518]:
x = tf.constant([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8])
#x = tf.constant([0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0])
x

<tf.Tensor: shape=(8,), dtype=float32, numpy=array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8], dtype=float32)>

In [519]:
input_layer = InputLayer(name = "InputLayer")
input_layer

In [520]:
res_input = input_layer(x)
res_input

<tf.Tensor: shape=(2, 14), dtype=float32, numpy=
array([[0.3, 0.4, 0.1, 0.2, 0.6, 0.2, 0.4, 0.4, 0.1, 0.1, 0.3, 0.1, 0.3,
        0.1],
       [0.5, 0.6, 0.6, 0.3, 0.7, 0.6, 0.8, 0.2, 0.4, 0.2, 0.4, 0.3, 0.7,
        0.5]], dtype=float32)>

In [521]:
input_layer.a

<tf.Variable 'InputLayer_activation:0' shape=(2, 14) dtype=float32, numpy=
array([[0.3, 0.4, 0.1, 0.2, 0.6, 0.2, 0.4, 0.4, 0.1, 0.1, 0.3, 0.1, 0.3,
        0.1],
       [0.5, 0.6, 0.6, 0.3, 0.7, 0.6, 0.8, 0.2, 0.4, 0.2, 0.4, 0.3, 0.7,
        0.5]], dtype=float32)>

In [522]:
hidden_layer = HiddenLayer(name = "HiddenLayer",weights_init_val=0.5, bias_init_val=0.0)
hidden_layer

In [523]:
res_hidden = hidden_layer(inputs = res_input, temperature = 1)
res_hidden

<tf.Tensor: shape=(1, 14), dtype=float32, numpy=
array([[0.59868765, 0.62245935, 0.5866176 , 0.5621765 , 0.65701044,
        0.59868765, 0.6456563 , 0.5744425 , 0.5621765 , 0.5374298 ,
        0.5866176 , 0.54983395, 0.62245935, 0.5744425 ]], dtype=float32)>

In [524]:
hidden_layer.w

<tf.Variable 'HiddenLayer/HiddenLayer_weights:0' shape=(2, 14) dtype=float32, numpy=
array([[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5,
        0.5],
       [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5,
        0.5]], dtype=float32)>

In [525]:
hidden_layer.a

<tf.Variable 'HiddenLayer/HiddenLayer_activation:0' shape=(1, 14) dtype=float32, numpy=
array([[0.59868765, 0.62245935, 0.5866176 , 0.5621765 , 0.65701044,
        0.59868765, 0.6456563 , 0.5744425 , 0.5621765 , 0.5374298 ,
        0.5866176 , 0.54983395, 0.62245935, 0.5744425 ]], dtype=float32)>

In [526]:
output_layer = OutputLayer(name = "OutputLayer",weights_init_val=0.59, bias_init_val=0.0)
output_layer

In [527]:
res_output = output_layer(inputs = res_hidden, temperature = 1)
res_output

<tf.Tensor: shape=(1, 14), dtype=float32, numpy=
array([[0.99249333, 0.99249333, 0.99249333, 0.99249333, 0.99249333,
        0.99249333, 0.99249333, 0.99249333, 0.99249333, 0.99249333,
        0.99249333, 0.99249333, 0.99249333, 0.99249333]], dtype=float32)>

In [528]:
output_layer.w

<tf.Variable 'OutputLayer/OutputLayer_weights:0' shape=(14, 14) dtype=float32, numpy=
array([[0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59,
        0.59, 0.59, 0.59],
       [0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59,
        0.59, 0.59, 0.59],
       [0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59,
        0.59, 0.59, 0.59],
       [0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59,
        0.59, 0.59, 0.59],
       [0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59,
        0.59, 0.59, 0.59],
       [0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59,
        0.59, 0.59, 0.59],
       [0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59,
        0.59, 0.59, 0.59],
       [0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59,
        0.59, 0.59, 0.59],
       [0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59,
        0.59, 0.59, 0.59],
     

In [529]:
output_layer.a

<tf.Variable 'OutputLayer/OutputLayer_activation:0' shape=(1, 14) dtype=float32, numpy=
array([[0.99249333, 0.99249333, 0.99249333, 0.99249333, 0.99249333,
        0.99249333, 0.99249333, 0.99249333, 0.99249333, 0.99249333,
        0.99249333, 0.99249333, 0.99249333, 0.99249333]], dtype=float32)>

In [530]:
error = tf.math.reduce_sum(tf.math.square(res_hidden - res_output))
error

<tf.Tensor: shape=(), dtype=float32, numpy=2.2692394>

### Test back propagation

In [531]:
#alpha = 1

In [532]:
#delta_Ao = res_output * (res_hidden - res_output) * (1 - res_output) + output_layer.deltaA * (res_hidden - res_output)
#delta_Ao

In [533]:
#res_hidden

In [534]:
#tf.transpose(res_hidden)

In [535]:
#delta_Ao

In [536]:
#delta_Wo = alpha * tf.matmul(tf.transpose(res_hidden),delta_Ao)   #dubbio scambiare righe e colonne
#delta_Wo

In [537]:
#output_layer.w.assign_add(delta_Wo)
#output_layer.w

In [538]:
#delta_Ah = res_hidden * (1 - res_hidden) * (tf.matmul(delta_Ao , output_layer.w)) # dubbio scambiare righe per colonne
#delta_Ah

In [539]:
#delta_Wh = tf.transpose(alpha * tf.transpose(delta_Ah) * res_input)
#delta_Wh

In [540]:
#hidden_layer.w.assign_add(delta_Wh)
#hidden_layer.w

---

# CUSTOM MODEL

In [541]:
class MQSelfReflexiveNetwork(tf.keras.Model):
  def __init__(self, learning_rate):
    super(MQSelfReflexiveNetwork, self).__init__(name='MQSelfReflexiveNetwork')

    self.input_layer = InputLayer(name = "InputLayer")
    self.h = HiddenLayer(name = "HiddenLayer",weights_init_val=0.5, bias_init_val=0.2)
    self.o = OutputLayer(name = "OutputLayer",weights_init_val=0.59, bias_init_val=0.2)
    self.temperature = tf.Variable(initial_value=1.0,trainable=False, name = "Temperature")
    self.error = tf.Variable(initial_value=0.0,trainable=False, name = "Error")
    self.alpha = learning_rate

  def call(self, input):
    ### FOWARD PASS
    res_input = self.input_layer(input)
    res_hidden = self.h(res_input, self.temperature)
    res_output = self.o(res_hidden, self.temperature)
    return tf.math.reduce_sum(tf.math.square(res_hidden - res_output))
  
  def train_step(self, input):
    ### CALL FOWARD PASS
    self.error.assign(self(input, training = True))

    ### BACK PROPAGATION
    # Compute temperature
    self.temperature.assign(1 - (1 / (1 + self.error) ))

    # Update outputs weights
    delta_Ao = self.o.a * (self.h.a - self.o.a) * (1 - self.o.a) + (self.o.deltaA * ((self.h.a - self.o.a)) )
    delta_Wo = self.alpha * tf.matmul(tf.transpose(self.h.a),delta_Ao)
    self.o.deltaA.assign(delta_Ao)
    self.o.w.assign_add(delta_Wo)

    ## Update hidden weights
    delta_Ah = self.h.a * (1 - self.h.a) * tf.matmul(self.o.deltaA, self.o.w)
    delta_Wh = self.alpha * delta_Ah * self.input_layer.a
    self.h.deltaA.assign(delta_Ah)
    self.h.w.assign_add(delta_Wh)

    return {
        "Error": tf.squeeze(self.error.value()),
        "Temperature": tf.squeeze(self.temperature.value()),
        "Hidden weights": tf.squeeze(self.h.w.value()),
        "Output weights": tf.squeeze(self.o.w.value())
        }

In [542]:
# Construct an instance of CustomModel
input = tf.constant([0.01, 0.00, 0.03, 0.224, 0.875, 0.266, 0.427, 0.168])
#input = tf.constant([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8])
model = MQSelfReflexiveNetwork(learning_rate=10)

In [543]:
model.compile()
model_history = model.fit(x = input, epochs=500)

Epoch 1/500
1/1 [==============================] - 0s 249ms/step - Error: 2.0959 - Temperature: 0.6770 - Hidden weights: 0.4906 - Output weights: 0.5776
Epoch 2/500
1/1 [==============================] - 0s 2ms/step - Error: 1.7098 - Temperature: 0.6310 - Hidden weights: 0.4930 - Output weights: 0.5815
Epoch 3/500
1/1 [==============================] - 0s 3ms/step - Error: 1.5334 - Temperature: 0.6053 - Hidden weights: 0.4920 - Output weights: 0.5799
Epoch 4/500
1/1 [==============================] - 0s 2ms/step - Error: 1.6949 - Temperature: 0.6289 - Hidden weights: 0.4922 - Output weights: 0.5802
Epoch 5/500
1/1 [==============================] - 0s 6ms/step - Error: 1.5575 - Temperature: 0.6090 - Hidden weights: 0.4919 - Output weights: 0.5799
Epoch 6/500
1/1 [==============================] - 0s 3ms/step - Error: 1.7824 - Temperature: 0.6406 - Hidden weights: 0.4918 - Output weights: 0.5797
Epoch 7/500
1/1 [==============================] - 0s 2ms/step - Error: 1.6713 - Temperature

In [544]:
df_model_history = pd.DataFrame(model_history.history)
df_model_history

,Error,Temperature,Hidden weights,Output weights
0,2.095945,0.676997,"[[0.4893418, 0.49323568, 0.4910166, 0.48361638...","[[0.5772953, 0.5770398, 0.5772503, 0.57886213,..."
1,1.709774,0.630965,"[[0.49137595, 0.49335203, 0.49557486, 0.486540...","[[0.5808043, 0.5827335, 0.5818547, 0.5830276, ..."
2,1.533425,0.605277,"[[0.4903459, 0.4901599, 0.49483308, 0.48654008...","[[0.57957643, 0.5810826, 0.57985085, 0.5812653..."
3,1.694932,0.628933,"[[0.4906643, 0.49018428, 0.49496633, 0.4866973...","[[0.5797136, 0.58142173, 0.5802914, 0.5815432,..."
4,1.557465,0.608988,"[[0.4906643, 0.48998415, 0.4949416, 0.48628518...","[[0.5793704, 0.5811758, 0.5799811, 0.5811936, ..."
...,...,...,...,...
495,1.253264,0.556199,"[[0.6101742, 0.86719054, 0.4378679, 0.45123273...","[[0.29151, 0.21228114, 0.042574108, 0.55627555..."
496,1.077196,0.518582,"[[0.6056598, 0.8546897, 0.43145907, 0.4400431,...","[[0.2834967, 0.2101266, -0.08979617, 0.5560704..."
497,1.349800,0.574432,"[[0.6128843, 0.85465294, 0.43959287, 0.4522823...","[[0.27793494, 0.20559537, -0.118887946, 0.5559..."
498,1.430749,0.588604,"[[0.61361104, 0.8951633, 0.4453258, 0.4686401,...","[[0.26164392, 0.20435157, -0.2074725, 0.555466..."


In [545]:
model.get_weights()

[array([[0.266, 0.168, 0.224, 0.01 , 0.03 , 0.01 , 0.168, 0.168, 0.224,
         0.224, 0.266, 0.224, 0.266, 0.224],
        [0.   , 0.03 , 0.03 , 0.266, 0.875, 0.03 , 0.427, 0.01 , 0.168,
         0.01 , 0.168, 0.266, 0.875, 0.   ]], dtype=float32),
 array([[0.60413635, 0.8863595 , 0.4446334 , 0.46808213, 0.40987736,
         0.6434324 , 0.8865068 , 0.9419669 , 0.3589669 , 0.44786894,
         0.52267957, 0.31478333, 0.48275065, 0.4020584 ],
        [0.49267486, 0.49683473, 0.4365766 , 0.5799505 , 0.58346444,
         0.3419758 , 0.26140758, 0.31752318, 0.86408323, 0.5666509 ,
         0.76733786, 0.7194443 , 0.86385125, 0.576036  ]], dtype=float32),
 array([[0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2,
         0.2]], dtype=float32),
 array([[-0.00356192, -0.00524035, -0.00030911, -0.00557952, -0.00135813,
         -0.0066064 , -0.0061942 , -0.00846337, -0.00410744, -0.00451227,
         -0.00451161, -0.00175757, -0.00331734, -0.00266265]],
       dtype=float32),


In [546]:
model.get_weights()[1][0]

array([0.60413635, 0.8863595 , 0.4446334 , 0.46808213, 0.40987736,
       0.6434324 , 0.8865068 , 0.9419669 , 0.3589669 , 0.44786894,
       0.52267957, 0.31478333, 0.48275065, 0.4020584 ], dtype=float32)

In [547]:
model.get_weights()[1][1]

array([0.49267486, 0.49683473, 0.4365766 , 0.5799505 , 0.58346444,
       0.3419758 , 0.26140758, 0.31752318, 0.86408323, 0.5666509 ,
       0.76733786, 0.7194443 , 0.86385125, 0.576036  ], dtype=float32)

In [548]:
dvf = abs(model.get_weights()[1][0] - model.get_weights()[1][1])
dvf

array([0.11146149, 0.3895248 , 0.00805679, 0.11186838, 0.17358708,
       0.30145657, 0.6250992 , 0.6244437 , 0.50511634, 0.11878198,
       0.24465829, 0.40466094, 0.3811006 , 0.17397758], dtype=float32)

In [549]:
model.get_weights()[0]

array([[0.266, 0.168, 0.224, 0.01 , 0.03 , 0.01 , 0.168, 0.168, 0.224,
        0.224, 0.266, 0.224, 0.266, 0.224],
       [0.   , 0.03 , 0.03 , 0.266, 0.875, 0.03 , 0.427, 0.01 , 0.168,
        0.01 , 0.168, 0.266, 0.875, 0.   ]], dtype=float32)

In [550]:
model.get_weights()[0][0,:]

array([0.266, 0.168, 0.224, 0.01 , 0.03 , 0.01 , 0.168, 0.168, 0.224,
       0.224, 0.266, 0.224, 0.266, 0.224], dtype=float32)

In [551]:
model.get_weights()[0][1,:]

array([0.   , 0.03 , 0.03 , 0.266, 0.875, 0.03 , 0.427, 0.01 , 0.168,
       0.01 , 0.168, 0.266, 0.875, 0.   ], dtype=float32)

In [552]:
dva = abs(model.get_weights()[0][0,:] - model.get_weights()[0][1,:])
dva

array([0.266     , 0.138     , 0.194     , 0.256     , 0.845     ,
       0.02      , 0.259     , 0.15799999, 0.05600001, 0.214     ,
       0.098     , 0.042     , 0.60899997, 0.224     ], dtype=float32)

In [553]:
dvf

array([0.11146149, 0.3895248 , 0.00805679, 0.11186838, 0.17358708,
       0.30145657, 0.6250992 , 0.6244437 , 0.50511634, 0.11878198,
       0.24465829, 0.40466094, 0.3811006 , 0.17397758], dtype=float32)

In [554]:
df_diff = pd.DataFrame({"dva":dva, "dvf":dvf})#, "dvf % dva":(dva/dvf)
df_diff

,dva,dvf
0,0.266,0.111461
1,0.138,0.389525
2,0.194,0.008057
3,0.256,0.111868
4,0.845,0.173587
5,0.020,0.301457
6,0.259,0.625099
7,0.158,0.624444
8,0.056,0.505116
9,0.214,0.118782


In [555]:
fig = px.bar(df_diff,title="Differences")
fig.show()

In [556]:
fig = px.line(df_model_history[["Error","Temperature"]])
fig.update_layout(title="Monitored variables",
                   xaxis_title='Epoch',
                   yaxis_title='',
                   hovermode='x unified')
fig.show()

---